In [11]:
from bs4 import BeautifulSoup #Beautiful Soup is a library that makes it easy to scrape information from web pages
import os
import pandas as pd
from unidecode import unidecode
import urllib.request #Functions for opening URLs
from pathlib import Path

In [12]:
list_review_text = {
    "contribution" : "Please describe the contribution of the paper",
    "strengths" : "Please list the main strengths of the paper",
    "weakness" : "Please list the main weaknesses of the paper",
    "clarity" : "Please rate the clarity and organization of this paper",
    "reproducibility" : "Please comment on the reproducibility of the paper",
    "detailed" : "Please provide detailed and constructive comments for the authors",
    "rate" : "Rate the paper on a scale of 1-8, 8 being the strongest",
    "justification" : "Please justify your recommendation.",
    "number of paper": "Number of papers in your stack",
    "ranking" : "What is the ranking of this paper in your review stack?",
    "confidence" : "Reviewer confidence",
    "rate rebuttal" : "[Post rebuttal] After reading the author’s rebuttal, state your overall opinion of the paper if it has been changed",
    "justification rebuttal" : "[Post rebuttal] Please justify your decision",
}

list_categories_str = [ "contribution", "strengths", "weakness", "reproducibility", "detailed", "justification" ]
list_categories_scores = ["clarity", "rate", "confidence", "rate rebuttal"]

columns_reviews = ["id", "category", "title" ,"review 1" ,"review 2" , "review 3", ]
columns_statistics = ["id", "category", "title" , "words1", "words2", "words3"]

In [13]:
def get_accepted_paper_list(year: str = "2023"):
    """
    Get the list of all html files from the chosen year
    """

    miccai_website_path = "https://conferences.miccai.org"
    path_online_list = miccai_website_path + f"/{year}/papers/"

    reponse = urllib.request.urlopen(path_online_list)
    contenu_web = reponse.read().decode('UTF-8')
    soup = BeautifulSoup(contenu_web, "html.parser")

    all = soup.find_all('a')
    paper_list = [(miccai_website_path + link.get('href')) for link in all if link.get('href').endswith('html')]

    return paper_list

In [14]:
def extract_reviews(paper, df_reviews, df_stats, df_scores, year="2023",  list_categories= "reproducibility"):
    """
    Extract the 3 first reviews from a given paper webpage
    """
    
    # open html webpage and extract with BeautifulSoup
    reponse = urllib.request.urlopen(paper)
    contenu_web = reponse.read().decode('UTF-8')
    soup = BeautifulSoup(contenu_web, "html.parser")

    if year == "2023" :
        paper_title = soup.find("title").get_text().rstrip("MICCAI 2023 - Accepted Papers, Reviews, Author Feedback").rstrip(' |')
    elif year == "2022" :
        paper_title = soup.find("title").get_text().rstrip("MICCAI 2022 - Accepted Papers and Reviews").rstrip(' |')
    paper_id = Path(paper).name[:13]

    try : 
        code_link = soup.find(lambda tag: tag.name == "a" and "https://git" in tag.text)
        code_link = code_link.get_text()
        code_link = code_link.replace("\n"," ")
    except:
        code_link = "N/A"
        pass
    
    df_reviews.loc[(paper_id, unidecode(paper_title)), ("code link", "code link")]=code_link
    
    for category in list_categories: 
        text = list_review_text[category]
        repro_reviews_paragraph = soup.find_all(lambda tag: tag.name == "li" and text in tag.text)
        repro_exact_text = soup.find(lambda tag: tag.name == "strong" and text in tag.text).get_text()
        
        i=0
        for review in repro_reviews_paragraph[:3]:
            i +=1
            tmp_review = unidecode(review.get_text().strip(repro_exact_text))
            tmp_review = tmp_review.strip("\n")
            tmp_review = tmp_review.replace("\n          \n"," ")
            tmp_review = tmp_review.replace("\t", " ")
            tmp_review = tmp_review.replace("\n\n\n\n"," ")
            tmp_review = tmp_review.replace("\n\n\n"," ")
            tmp_review = tmp_review.replace("\n\n"," ")
            tmp_review = tmp_review.replace("\n"," ")
            tmp_review = tmp_review.replace("\t", " ")

            if category in list_categories_str:
                df_reviews.loc[(paper_id, unidecode(paper_title)), (category, f"review {i}")]=tmp_review
                df_stats.loc[(paper_id, unidecode(paper_title)), (category, f"review {i}")]=len(str(tmp_review).split())
            elif category in list_categories_scores:
                df_scores.loc[(paper_id, unidecode(paper_title)), (category, f"review {i}")]=tmp_review

In [15]:

def extract_paragraph(paper_list, year= "2023"):
    """
    Extract the 3 first reviews of all paper from the given list.
    
    Return 3 DatFrame:
        - df_reviews: DataFrame containing the 3 first reviews of all papers
        - df_stats: DataFrame containing the number of words for each review of df_reviews
        - df_scores: DataFrame containing the scores given by the 3 first reviewers of all papers.
    """
    iterables_str = [list_categories_str, ["review 1", "review 2", "review 3"]]
    iterables_score = [list_categories_scores, ["review 1", "review 2", "review 3"]]

    index_line_str = pd.MultiIndex.from_product(iterables_str, names=["category", "review"])
    index_line_score = pd.MultiIndex.from_product(iterables_score, names=["category", "review"])
    index_column = pd.MultiIndex.from_product( [[], []], names=["id", "title"])

    df_reviews = pd.DataFrame(index = index_column, columns=index_line_str)
    df_stats = pd.DataFrame(index = index_column, columns=index_line_str)
    df_scores = pd.DataFrame(index = index_column, columns=index_line_score)

    for i, paper in enumerate(paper_list):
        print(f"Processing paper {i + 1}/{len(paper_list)}", end='\r', flush=True)
        list_categories = list_categories_str + list_categories_scores
        extract_reviews(paper, df_reviews, df_stats, df_scores, year, list_categories)

    return df_reviews, df_stats, df_scores

In [16]:
def get_repro_copy_paste(df_all_reviews, output_directory= "../results", threshold:int  = 10 ):
    """
    Find reviewers that have copy/paste the reproducibility review in another category.
    
    """
    from copy import copy
    df_all_reviews_wo_copy_paste = copy(df_all_reviews)
    df_bad_reviews = pd.DataFrame(columns=columns_reviews)
    df_bad_reviews.set_index(["id", "category"], inplace= True)
    
    for id, id_df in df_all_reviews.groupby(level=0):
        for _, title in id_df.index.values:
            for category in list_categories_str:
                if category != "reproducibility" :
                    for i in range(1,3):
                        repro = id_df.loc[(id,title), ("reproducibility", f"review {i}")]
                        cate = id_df.loc[(id, title), (category, f"review {i}")]
                        if len(str(repro)) > threshold : 
                            if str(repro) in str(cate):
                                df_bad_reviews.loc[(id, title), (category, f"review {i}")] = id_df.loc[(id, title), (category, f"review {i}")]
                                try :
                                    df_all_reviews_wo_copy_paste.drop((id, title), inplace=True)
                                except:
                                    pass
                                
    df_all_reviews_wo_copy_paste.to_csv(os.path.join(output_directory ,f'reviews_wo_copy_paste_{threshold}.csv'), index = True, sep="\t", encoding='utf-8')
    df_bad_reviews.to_csv(os.path.join(output_directory ,f'reviews_copy_paste_{threshold}.csv'), index = True, sep="\t", encoding='utf-8')

In [17]:
def count_checklist(df_all_reviews, output_directory= "../results", category = "reproducibility"):
    """
    Find reviewers that have mentionned the word 'cheklist' in the reproducibility review.

    """
    df_checklist  = pd.DataFrame(columns=columns_reviews)
    df_checklist.set_index(["id", "category"], inplace= True)
    for id, id_df in df_all_reviews.groupby(level=0):
        for _, title in id_df.index.values:
            for i in range(1,4):
                review = str(df_all_reviews.loc[(id, title), (category, f"review {i}")])

                if ("check-list" in review) or ("checklist" in review) or ("check list" in review):
                    df_checklist.loc[(id, title), (category, f"review {i}")] = df_all_reviews.loc[(id, title), (category, f"review {i}")]

    df_checklist.to_csv(os.path.join(output_directory ,f'{category}_checklist_reviews.csv'), index = True, sep="\t", encoding='utf-8')

In [18]:
def create_rating_excel(df_all_reviews, output_directory="../results"):
    """
    Save the excel file with the reproducibility reviews to create the rating file
    """
    df_repro_excel = df_all_reviews.loc[:, ("reproducibility")]
    df_repro_excel.to_excel(os.path.join('../rating' ,f'reviews_reproducibility_{year_}.xlsx'))

In [19]:
# Note extraction takes 5-10 minutes

# Default year is 2023
# To change to another year, please modify the variable below
year_ = "2023"

# Get the list of HTML pages for the different papers
paper_list = get_accepted_paper_list(year=year_)

# Set output directories and files for the extraction
output_directory = Path(f"../miccai{year_}")
if not output_directory.is_dir():
    os.mkdir(output_directory)
path_all_reviews = output_directory / 'reviews.csv'
path_all_stats = output_directory / 'count_words.csv'
path_all_scores = output_directory / 'scores.csv'

# Skip the extraction if the files already exist
# If you want to rerun the extraction, remove the directory "miccaiYYYY" where YYYY is the year  
if (not path_all_reviews.is_file()) or (not path_all_stats.is_file()) or (not path_all_scores.is_file()):
    print(f"Extract reviews and count word for year {year_}")
    df_all_reviews, df_all_stats, df_all_scores = extract_paragraph(paper_list, year_)
    df_all_scores.to_csv(path_all_scores, index = True, sep="\t", encoding='utf-8')
    df_all_reviews.to_csv(path_all_reviews, index = True, sep="\t", encoding='utf-8')
    df_all_stats.to_csv(path_all_stats, index = True, sep="\t", encoding='utf-8')

else:
    print(f"Files already exist")
    print(f"Skipping extraction and importing existing csv from {output_directory}")
    df_all_reviews = pd.read_csv(path_all_reviews, sep= "\t",  header=[0, 1], index_col=[0,1], skip_blank_lines=True)
    df_all_stats = pd.read_csv(path_all_stats, sep= "\t",  header=[0, 1], index_col=[0,1], skip_blank_lines=True)



Extract reviews and count word for year 2023


In [20]:
print("Identify reproducibility reviews which contain copy/paste from other parts (need to have more than 10 consecutive words in common)")
get_repro_copy_paste(df_all_reviews=df_all_reviews, output_directory=output_directory)

print("Count the number of reviews that mention the checklist")
count_checklist(df_all_reviews=df_all_reviews, output_directory=output_directory, category="reproducibility")

print("Create excel files for human rating")
create_rating_excel(df_all_reviews= df_all_reviews, output_directory= output_directory)

Identify reproducibility reviews which contain copy/paste from other parts (need to have more than 10 consecutive words in common)


/var/folders/rf/9f5s04x100n1rl5c_b92rsh8000kc6/T/ipykernel_16499/1210693690.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'The code is not provided. The method is built upon the training segmentation label.' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_bad_reviews.loc[(id, title), (category, f"review {i}")] = id_df.loc[(id, title), (category, f"review {i}")]
/var/folders/rf/9f5s04x100n1rl5c_b92rsh8000kc6/T/ipykernel_16499/1210693690.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Insufficient Dataset Description: The authors have not provided an adequate description of the dataset utilized in the study, which may hinder the reader's understanding of the context and the applicability of the proposed method. Furthermore, the lack of information regarding the preprocessing steps implement

Count the number of reviews that mention the checklist
Create excel files for human rating


/var/folders/rf/9f5s04x100n1rl5c_b92rsh8000kc6/T/ipykernel_16499/4058375330.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Reproducibility should be possible, as training and evaluation code is provided (based on the checklist).' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_checklist.loc[(id, title), (category, f"review {i}")] = df_all_reviews.loc[(id, title), (category, f"review {i}")]
